In [1]:
import os

In [2]:
%pwd

'/workspaces/Text-Summarizer/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/workspaces/Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
import sys
sys.path.append("src")

In [ ]:
from textSummarizer.constants import *

In [ ]:
!pip install pyyaml

  Using cached aiohttp-3.10.11-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Preparing metadata (setup.py) ... done
  Using cached aiohappyeyeballs-2.4.4-py3-none-any.whl.metadata (6.1 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached frozenlist-1.5.0-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
  Using cached multidict-6.1.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.0 kB)
  Using cached yarl-1.15.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (56 kB)
  Using cached async_timeout-5.0.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached propcache-0.2.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.3 MB/s 

In [25]:
!pip uninstall box
!pip install python-box
from box import ConfigBox
from box.exceptions import BoxValueError


Found existing installation: box 0.1.5
Uninstalling box-0.1.5:
  Would remove:
    /opt/conda/envs/textS/lib/python3.8/site-packages/box-0.1.5.dist-info/*
    /opt/conda/envs/textS/lib/python3.8/site-packages/box/*
    /opt/conda/envs/textS/lib/python3.8/site-packages/waivek/*
Proceed (Y/n)? ERROR: Operation cancelled by user
^C
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 24.5 MB/s eta 0:00:00


In [27]:
!pip install ensure

In [28]:
from textSummarizer.utils.common import read_yaml, create_directories

In [29]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [30]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [42]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [33]:
!pip install e

  Preparing metadata (setup.py) ... done
  Created wheel for e: filename=e-1.4.5-py3-none-any.whl size=2795 sha256=bc8d900c8c49a38e57b659090c073bc531ef37965d608f87f4612bdf6f875ee2
  Stored in directory: /home/codespace/.cache/pip/wheels/4e/79/9d/507a7eeaeae1d2ba45b52cb04e004838b5f2126f9c577557ab
Successfully built e


In [37]:
print("Reading YAML from: /workspaces/Text-Summarizer/config/config.yaml")

Reading YAML from: /workspaces/Text-Summarizer/config/config.yaml


In [41]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2025-07-05 00:32:21,742: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-07-05 00:32:21,746: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-05 00:32:21,759: INFO: common: created directory at: artifacts]
[2025-07-05 00:32:21,761: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-05 00:32:22,691: INFO: 1434958058: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 24C8:1E034B:A7916E:BE97FB:68687294
Accept-Ranges: bytes
Date: Sat, 05 Jul 2025 00:32:22 GMT
Via: 1.1 varnish
X-Served-By: cache-ams2100100-AMS
X-Cache: